#### 导入包与数据预处理

In [5]:
options(warn=-1)

In [6]:
library(lubridate)
library(xts)
library(quantmod)
library(Tushare)
library(TSA)
library(fUnitRoots)
library(urca)
library(fBasics)
library(forecast)
library(FinTS)
library(fGarch)
library(rugarch)
library(PerformanceAnalytics)
library(quantreg)

In [7]:
api<-pro_api(token="731ee2eafa47ae6b8c98993e8b30001b51bd3085cd69d63870e745c9")
xcdl<-api(api_name ='daily',ts_code ='600505.SH',start_date='20170101',end_date='20220101')
time<-as.Date(ymd(xcdl$trade_date))
new_time<-as.Date(ymd(xcdl$trade_date[1:243]))
xcdl_close_price<-xts(xcdl$close,order.by=time)
xcdl_logreturn<-dailyReturn(xcdl_close_price,type ="log")[-1]#计算红相股份简单收益率

# 运用选取的股票进行在险价值的测算与分析

## 分别运用 delta normal 方法、历史模拟法、 Monte Carlo 模拟法计算该只股票的VaR 和 ES 值（考虑尾部概率为 5% 和 1% 两种情形）

In [8]:
#基于delta normal
#计算VaR
var_xcdl_1<-abs(qnorm(0.05,mean(xcdl_logreturn),sd(xcdl_logreturn)))
#计算ES
sloss<-sort( -as.numeric(xcdl_logreturn ),decreasing=F)
es_1<-sum(sloss[sloss>=var_xcdl_1])/length(sloss[sloss>var_xcdl_1])
#计算VAR
var_xcdl_2<-abs(mean(xcdl_logreturn)+qnorm(0.95)*sd(xcdl_logreturn))
#计算ES
es_2<-sum(sloss[sloss>=var_xcdl_2])/length(sloss[sloss>var_xcdl_2])
#基于历史模拟法
xcdl_sort=sort(xcdl_logreturn)
var_xcdl_3<-quantile(-xcdl_sort,0.95,low=T)
VaR(xcdl_sort,p =0.95,method="historical")
es_3<-sum(sloss[sloss>=var_xcdl_3])/length(sloss[sloss>var_xcdl_3])
#基于蒙特卡洛
xcdl_mt<-rnorm(10000000,mean(xcdl_logreturn),sd(xcdl_logreturn))
var_xcdl_4<-quantile(-xcdl_mt,0.95,low=T)
es_4<-sum(sloss[sloss>=var_xcdl_4])/length(sloss[sloss>var_xcdl_4])
#基于delta normal
#计算VaR
var_1_xcdl_1<-abs(qnorm(0.01,mean(xcdl_logreturn),sd(xcdl_logreturn)))
#计算ES
sloss<-sort( -as.numeric(xcdl_logreturn ),decreasing=F)
es_1_1<-sum(sloss[sloss>=var_1_xcdl_1])/length(sloss[sloss>var_1_xcdl_1])
#计算VAR
var_1_xcdl_2<-abs(mean(xcdl_logreturn)+qnorm(0.99)*sd(xcdl_logreturn))
#计算ES
es_1_2<-sum(sloss[sloss>=var_1_xcdl_2])/length(sloss[sloss>var_1_xcdl_2])
#基于历史模拟法
xcdl_sort=sort(xcdl_logreturn)
var_1_xcdl_3<-quantile(-xcdl_sort,0.99,low=T)
VaR(xcdl_sort,p =0.99,method="historical")
es_1_3<-sum(sloss[sloss>=var_1_xcdl_3])/length(sloss[sloss>var_1_xcdl_3])
#基于蒙特卡洛
xcdl_mt<-rnorm(10000000,mean(xcdl_logreturn),sd(xcdl_logreturn))
var_1_xcdl_4<-quantile(-xcdl_mt,0.99,low=T)
es_1_4<-sum(sloss[sloss>=var_1_xcdl_4])/length(sloss[sloss>var_1_xcdl_4])

,daily.returns
VaR,-0.04390758


,daily.returns
VaR,-0.07180222


## 运用风险度量制计算该只股票 1 天和 10 天的 VaR 和 ES 值（考虑尾部概率为 5% 和1% 两种情形）

In [9]:
spec1<-ugarchspec(variance.model =list(model= "iGARCH", garchOrder =c(1,1)), mean.model = list(armaOrder =c(0,0), include.mean = F),distribution.model ="norm", fixed.pars =list(omega=0 ,alpha1=0.06))
forecast1<-ugarchforecast(spec1, xcdl_logreturn,n.roll =0,n.ahead=1,out.sample=0)
#0.95 一天
var_xcdl_5<-qnorm(0.95)*forecast1@forecast$sigmaFor
es_5<-dnorm(qnorm (0.95))*forecast1@forecast$sigmaFor/0.05
# 0.95 10天
var_xcdl_5_1<-qnorm(0.95)*forecast1@forecast$sigmaFor*sqrt(10)
es_5_1<-dnorm(qnorm (0.95))*forecast1@forecast$sigmaFor*sqrt(10)/0.05
#0.99 1天
var_xcdl_6<-qnorm(0.99)*forecast1@forecast$sigmaFor
es_6<-dnorm(qnorm (0.99))*forecast1@forecast$sigmaFor/0.05
#0.99 10天
var_xcdl_6_1<-qnorm(0.99)*forecast1@forecast$sigmaFor*sqrt(10)
es_6_1<-dnorm(qnorm (0.99))*forecast1@forecast$sigmaFor*sqrt(10)/0.05

## 运用 GARCH 模型计算并比较不同条件分布假设下计算该只股票 1 天和 10 天的 VaR和 ES 值（考虑尾部概率为 5% 和 1% 两种情形）

In [12]:
#正态分布
xcdl_garch<-garchFit(formula=~garch (1,1),data =xcdl_logreturn, cond.dist =c("norm"),trace= F,include.mean =F)
xcdl_pre_1<-predict(xcdl_garch,n.ahead=1)
#0.95 1天
var_xcdl_7<-xcdl_pre_1$meanForecast+xcdl_pre_1$standardDeviation*qnorm(0.95)
es_7<-xcdl_pre_1$meanForecast+xcdl_pre_1$standardDeviation*dnorm(qnorm(0.95))/(1-0.95)
#0.99 1天
var_xcdl_8<-xcdl_pre_1$meanForecast+xcdl_pre_1$standardDeviation*qnorm(0.99)
es_8<-xcdl_pre_1$meanForecast+xcdl_pre_1$standardDeviation*dnorm(qnorm(0.99))/(1-0.99)
#标准t分布
xcdl_garch_1<-garchFit(formula=~garch (1,1),data =xcdl_logreturn, cond.dist =c("std"),trace= F,include.mean =F)
xcdl_pre_3<-predict(xcdl_garch_1,n.ahead=1)
#0.95 1天
var_xcdl_9<-xcdl_pre_3$meanForecast+xcdl_pre_3$standardDeviation*qnorm(0.95)
es_9<-xcdl_pre_3$meanForecast+xcdl_pre_3$standardDeviation*dnorm(qnorm(0.95))/(1-0.95)
#0.99 1天
var_xcdl_10<-xcdl_pre_3$meanForecast+xcdl_pre_3$standardDeviation*qnorm(0.99)
es_10<-xcdl_pre_3$meanForecast+xcdl_pre_3$standardDeviation*dnorm(qnorm(0.99))/(1-0.99)
#多期
#标准t分布
source("SimGarcht.R")
vol<-volatility(xcdl_garch_1)
a1<-c(xcdl_garch_1@fit$coef["omega"],xcdl_garch_1@fit$coef["alpha1"])
b1<-xcdl_garch_1@fit$coef["beta1"];
mu=0
ini=c(coredata(xcdl_logreturn[1216]),vol[1216])
simgarcht_xcdl_1<-SimGarcht(h=10,mu=mu,alpha = a1,b1=b1,df=xcdl_garch_1@fit$coef["shape"],ini=ini,nter = 30000)
sim_r_xcdl<-simgarcht_xcdl_1$rtn
var_xcdl_11<-quantile(-sim_r_xcdl,0.95)
tail_sh<-c(1:30000)[ -sim_r_xcdl >var_xcdl_11]
es_11<-mean( -sim_r_xcdl [tail_sh])
var_xcdl_12<-quantile(-sim_r_xcdl,0.99)
tail_sh<-c(1:30000)[ -sim_r_xcdl >var_xcdl_12]
es_12<-mean( -sim_r_xcdl [tail_sh])
#正态分布
vol<-volatility(xcdl_garch)
a1<-c(xcdl_garch@fit$coef["omega"],xcdl_garch@fit$coef["alpha1"])
b1<-xcdl_garch@fit$coef["beta1"];
mu=0
ini=c(coredata(xcdl_logreturn[1216]),vol[1216])
simgarcht_xcdl_1<-SimGarcht(h=10,mu=mu,alpha = a1,b1=b1,ini=ini,nter = 30000)
sim_r_xcdl<-simgarcht_xcdl_1$rtn
var_xcdl_17<-quantile(-sim_r_xcdl,0.95)
tail_sh<-c(1:30000)[ -sim_r_xcdl >var_xcdl_17]
es_17<-mean( -sim_r_xcdl [tail_sh])
var_xcdl_18<-quantile(-sim_r_xcdl,0.99)
tail_sh<-c(1:30000)[ -sim_r_xcdl >var_xcdl_18]
es_18<-mean( -sim_r_xcdl [tail_sh])

## 运用分位数回归的方法计算并比较不同条件分布假设下以交易量为解释变量，计算该只股票 1 天和 10 天的 VaR 和 ES 值（考虑尾部概率为 5% 和 1%)

In [13]:
xcdl_volume<-as.numeric(xcdl$vol)
xcdl_volume<-(log(xcdl_volume))[-1]
XCDL<-data.frame(xcdl_logreturn,xcdl_volume)
fit1<-rq(formula = xcdl_logreturn~xcdl_volume,tau = 0.05,data = XCDL)
#0.95 1天
xcdl_pre<-api(api_name ='daily',ts_code ='600505.SH',start_date='20220101',end_date='20220201')
xcdl_volume_pre<-as.numeric(xcdl_pre$vol)
xcdl_volume_pre<-(log(xcdl_volume_pre))[-1]
vol_1217<-xcdl_volume_pre[1]
var_xcdl_13<-abs(fit1$coefficients[1]+fit1$coefficients[2]*vol_1217)
es_13<-sum(sloss[sloss>=var_xcdl_13])/length(sloss[sloss>var_xcdl_13])
#0.95 10天
vol_1226<-xcdl_volume_pre[10]
var_xcdl_14<-abs(fit1$coefficients[1]+fit1$coefficients[2]*vol_1226)
es_14<-sum(sloss[sloss>=var_xcdl_14])/length(sloss[sloss>var_xcdl_14])
fit2<-rq(formula = xcdl_logreturn~xcdl_volume,tau = 0.01,data = XCDL)
#0.99 1天
var_xcdl_15<-abs(fit2$coefficients[1]+fit2$coefficients[2]*vol_1217)
es_15<-sum(sloss[sloss>=var_xcdl_15])/length(sloss[sloss>var_xcdl_15])
#0.99 10天
var_xcdl_16<-abs(fit2$coefficients[1]+fit2$coefficients[2]*vol_1226)
es_16<-sum(sloss[sloss>=var_xcdl_16])/length(sloss[sloss>var_xcdl_16])

## 讨论上述根据多种方法计算得到的结果存在差异的原因

In [37]:
VaR_1tian_0.95<-c(var_xcdl_1,var_xcdl_2,var_xcdl_3,var_xcdl_4,
                  var_xcdl_5,var_xcdl_7,var_xcdl_9,var_xcdl_13)
VaR_1tian_0.99<-c(var_1_xcdl_1,var_1_xcdl_2,var_1_xcdl_3,var_1_xcdl_4,
                  var_xcdl_6,var_xcdl_8,var_xcdl_10,var_xcdl_15)
VaR_10tian_0.95<-c(var_xcdl_5_1,var_xcdl_17,var_xcdl_11,var_xcdl_14)
VaR_10tian_0.99<-c(var_xcdl_6_1,var_xcdl_18,var_xcdl_12,var_xcdl_16)
VaR_1tian_0.95<-setNames(VaR_1tian_0.95,name1)
VaR_1tian_0.99<-setNames(VaR_1tian_0.99,name1)
VaR_10tian_0.95<-setNames(VaR_10tian_0.95,name2)
VaR_10tian_0.99<-setNames(VaR_10tian_0.99,name2)
ES_1tian_0.95<-c(es_1,es_2,es_3,es_4,es_5,es_7,es_9,es_13)
ES_1tian_0.99<-c(es_1_1,es_1_2,es_1_3,es_1_4,es_6,es_8,es_10,es_15)
ES_10tian_0.95<-c(es_5_1,es_17,es_11,es_14)
ES_10tian_0.99<-c(es_6_1,es_18,es_12,es_16)
ES_1tian_0.95<-setNames(ES_1tian_0.95,name1)
ES_1tian_0.99<-setNames(ES_1tian_0.99,name1)
ES_10tian_0.95<-setNames(ES_10tian_0.95,name2)
ES_10tian_0.99<-setNames(ES_10tian_0.99,name2)

In [38]:
dat1<-cbind(VaR_1tian_0.95,VaR_1tian_0.99)
dat2<-cbind(ES_1tian_0.95,ES_1tian_0.99)
dat3<-cbind(VaR_10tian_0.95,VaR_10tian_0.99)
dat4<-cbind(ES_10tian_0.95,ES_10tian_0.99)

In [39]:
dat1

,VaR_1tian_0.95,VaR_1tian_0.99
delta normal方法1,0.04480847,0.06337870
delta normal方法2,0.04483370,0.06340393
历史模拟法,0.04390758,0.07180222
Monte Cralo,0.04480512,0.06336238
风险度量制,0.07760877,0.10976356
GARCH模型-Norm,0.07836319,0.11083056
GARCH模型-Std,0.09127142,0.12908692
分位数回归法,0.03602478,0.06559182


In [40]:
dat2

,ES_1tian_0.95,ES_1tian_0.99
delta normal方法1,0.06319050,0.08346417
delta normal方法2,0.06319050,0.08346417
历史模拟法,0.06257801,0.09363884
Monte Cralo,0.06319050,0.08346417
风险度量制,0.09732453,0.02515044
GARCH模型-Norm,0.09827060,0.12697464
GARCH模型-Std,0.11445805,0.14789030
分位数回归法,0.05674244,0.08444080


In [35]:
dat3

,VaR_10tian_0.95,VaR_10tian_0.99
风险度量制,0.2454205,0.34710286
GARCH模型-Norm,0.2179042,0.34686023
GARCH模型-Std,0.2455758,0.44079059
分位数回归法,0.0369250,0.06725244


In [36]:
dat4

,ES_10tian_0.95,ES_10tian_0.99
风险度量制,0.3077672,0.07953269
GARCH模型-Norm,0.3001773,0.44147328
GARCH模型-Std,0.3771026,0.62990207
分位数回归法,0.0575210,0.08766893


### VaR差异比较
- 在一天 尾部概率为0.05的情况下：delta normal方法、历史模拟法、Monte Cralo方法计算的VaR相差很小，风险度量制的计算结果高于其他方法，GARCH模型在正态分布情况下与风险度量制结果差异小，最大值是GARCH模型在标准t分布情况下计算得到的，分位数回归法VaR的值最小
- 在一天 尾部概率为0.01的情况下：delta normal方法、历史模拟法、Monte Cralo方法计算、分位数回归的VaR相差很小，风险度量制的计算结果高于其他方法，GARCH模型计算结果差异不大，但高于其他方法结果
- 在多期的情况下，风险度量制，GARCH模型的计算结果差异很小，但分位数回归计算的结果要小得多

### ES差异比较
- 在一天 尾部概率为0.05的情况下：delta normal方法、历史模拟法、Monte Cralo方法计算的ES相差很小,风险度量制的计算结果高于其他方法，GARCH模型在正态分布情况下与风险度量制结果差异小，最大值是GARCH模型在标准t分布情况下计算得到的，分位数回归法ES的值最小
- 在一天 尾部概率为0.01的情况下：delta normal方法、分位数回归法、Monte Cralo方法计算所得的ES也是较为接近的历史模拟法略高于上述三种方法，风险度量制的计算结果最小，GARCH模型计算结果差异不大，但大于其他方法结果，最大值是GARCH模型在标准t分布情况下计算得到的
- 在十天 尾部概率为0.05的情况下：风险度量制，GARCH模型的计算结果差异很小，但分位数回归计算的结果要小得多
- 在十天 尾部概率为0.01的情况下：风险度量制，分位数回归法的计算结果差异很小，但GARCH计算的结果要大得多

### 产生差异可能的原因：
- 1、证券收益率的分布可能不具有厚尾效应
- 2、模型对于收益分布的假设不正确
- 3、GARCH模型不充分，模型拟合效果差
- 4、分位数回归在q接近1是变得难以估计，对于一个很大的q缺乏观测值，回归结果变得无信息 